In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import tensorflow as tf
from tensorflow.keras import layers, Model
import random
from sklearn.metrics import accuracy_score
from numpy.linalg import norm
import time

In [2]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),      # lật ngang ngẫu nhiên
    layers.RandomRotation(0.1),           # quay ±10%
    layers.RandomZoom(0.1),               # zoom ngẫu nhiên trong khoảng ±10%
    layers.RandomTranslation(0.1, 0.1),    # dịch chuyển ±10% chiều cao & rộng
    layers.RandomContrast(0.1),           # thay đổi độ tương phản ±10%
])

In [3]:
def build_embedding_model(input_shape=(160,160,3), embedding_dim=128):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)

    embeddings = layers.Dense(embedding_dim, activation=None, name='embeddings')(x)
    embeddings = layers.Lambda(lambda t: tf.math.l2_normalize(t, axis=1))(embeddings)

    return Model(inputs, embeddings, name='embedding_model')

In [4]:
def build_classification_model(embedding_model, num_classes):
    x = embedding_model.output
    outputs = layers.Dense(num_classes, activation='softmax', name='softmax')(x)
    return Model(embedding_model.input, outputs, name='classification_model')


In [5]:
train_dir = "/Users/hqpl/Downloads/archive/train" # Đường dẫn đến thư mục
val_dir = "/Users/hqpl/Downloads/archive/val" # Đường dẫn đến thư mục


In [6]:
# 1) Khởi tạo embedding model
emb_model = build_embedding_model(input_shape=(160,160,3), embedding_dim=128)
pre_weights = emb_model.get_weights()

In [7]:
# 2) Tạo classification model
num_classes = len(os.listdir(train_dir))  # số người
cls_model = build_classification_model(emb_model, num_classes)

In [8]:
# 3) Compile với Sparse Categorical Crossentropy
cls_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [9]:
# 4) Load data bằng image_dataset_from_directory
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, image_size=(160,160), batch_size=32, label_mode='int', shuffle=True, seed=123
).map(
    lambda x, y: (data_augmentation(x), y),
    num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,   image_size=(160,160), batch_size=32, label_mode='int', shuffle=False)

Found 176396 files belonging to 480 classes.
Found 21294 files belonging to 60 classes.


In [10]:
# 5) Huấn luyện
cls_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint("best_cls.h5", save_best_only=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=3)
    ]
)

Epoch 1/10
5513/5513 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.0066 - loss: 6.1324

5513/5513 ━━━━━━━━━━━━━━━━━━━━ 1093s 198ms/step - accuracy: 0.0066 - loss: 6.1324 - val_accuracy: 0.0000e+00 - val_loss: 6.2016 - learning_rate: 1.0000e-04
Epoch 2/10
5513/5513 ━━━━━━━━━━━━━━━━━━━━ 1098s 199ms/step - accuracy: 0.0150 - loss: 5.8323 - val_accuracy: 4.6962e-05 - val_loss: 6.3689 - learning_rate: 1.0000e-04
Epoch 3/10
5513/5513 ━━━━━━━━━━━━━━━━━━━━ 1102s 200ms/step - accuracy: 0.0204 - loss: 5.5725 - val_accuracy: 1.4088e-04 - val_loss: 6.4248 - learning_rate: 1.0000e-04
Epoch 4/10
 272/5513 ━━━━━━━━━━━━━━━━━━━━ 17:12 197ms/step - accuracy: 0.0252 - loss: 5.4248

KeyboardInterrupt: 

In [26]:
post_weights = emb_model.get_weights()
changed = False
for idx, (w_pre, w_post) in enumerate(zip(pre_weights, post_weights)):
    if not np.allclose(w_pre, w_post):
        print(f"Layer {idx} has changed the weight (diff normal = {np.linalg.norm(w_pre - w_post):.6f})")
        changed = True

if not changed:
    print("No weight changes — may be training not running or weights not updated.")
else:
    print("✅ Emb_model weights has been updated after training.")

Layer 0 has changed the weight (diff normal = 0.356102)
Layer 1 has changed the weight (diff normal = 0.169268)
Layer 2 has changed the weight (diff normal = 0.148779)
Layer 3 has changed the weight (diff normal = 0.111999)
Layer 4 has changed the weight (diff normal = 199.525986)
Layer 5 has changed the weight (diff normal = 6075.502441)
Layer 6 has changed the weight (diff normal = 1.643861)
Layer 7 has changed the weight (diff normal = 0.149267)
Layer 8 has changed the weight (diff normal = 0.097590)
Layer 9 has changed the weight (diff normal = 0.522394)
Layer 10 has changed the weight (diff normal = 3.026129)
Layer 11 has changed the weight (diff normal = 5.705964)
Layer 12 has changed the weight (diff normal = 2.167116)
Layer 13 has changed the weight (diff normal = 0.297375)
Layer 14 has changed the weight (diff normal = 0.153807)
Layer 15 has changed the weight (diff normal = 0.269819)
Layer 16 has changed the weight (diff normal = 3.188481)
Layer 17 has changed the weight (dif

In [12]:
# Load weights tốt nhất vào classification model
cls_model.load_weights("best_cls.h5")

In [13]:
emb_model.save("face_embedding_model_64.keras")
emb_model.save("face_embedding_model_64.h5")

In [14]:
def preprocess_image(img_path, target_size=(160,160)):
    # Đọc ảnh BGR và chuyển sang RGB
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    # Resize
    img = cv2.resize(img, target_size)
    # Chuyển sang float và normalize về [-1,1]
    img = img.astype(np.float32)
    img = (img / 127.5) - 1.0
    # Thêm batch dimension
    img = np.expand_dims(img, axis=0)  # shape (1, H, W, C)
    return img

In [29]:
# Đường dẫn tới ảnh muốn test
img_path = "train/n000004/0003_01.jpg"
x = preprocess_image(img_path)
# predict trả về array shape (1, embedding_dim)
emb_vector = emb_model.predict(x)
# bỏ batch dimension nếu cần
emb_vector = emb_vector[0]
print("Embedding vector shape:", emb_vector.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Embedding vector shape: (128,)


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

# Giả sử có hai ảnh test
emb1 = emb_model.predict(preprocess_image("/Users/hqpl/Downloads/archive/val/n000001/0001_01.jpg"))[0]
emb2 = emb_model.predict(preprocess_image("/Users/hqpl/Downloads/archive/val/n000001/0002_01.jpg"))[0]
sim = cosine_similarity(emb1, emb2)
print(f"Cosine similarity = {sim:.3f}")
# Thresholding: ví dụ sim > 0.5 là cùng người (tùy kiểm định)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Cosine similarity = 0.850


In [22]:
# Giả sử emb_model đã được load hoặc khởi tạo
# emb_model = tf.keras.models.load_model("face_embedding_model")

# Tạo một batch giả để đo tốc độ (vd. batch_size=32)
batch_size = 32
dummy_batch = np.random.randn(batch_size, 160, 160, 3).astype(np.float32)

# 1) Warm‑up
_ = emb_model.predict(dummy_batch)

# 2) Đo inference
n_runs = 100
start = time.time()
for _ in range(n_runs):
    _ = emb_model.predict(dummy_batch)
end = time.time()

avg_time = (end - start) / n_runs
print(f"Average inference time per batch of {batch_size}: {avg_time:.4f} seconds")
print(f"  => Per image: {avg_time / batch_size:.4f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━

In [25]:
# 2) Chuẩn bị list ảnh
img_dir = "/Users/hqpl/Downloads/archive/val/n000001"
img_paths = [
    os.path.join(img_dir, fn)
    for fn in os.listdir(img_dir)
    if fn.lower().endswith(('.jpg','.png','.jpeg'))
]

# 3) Hàm tiền xử lý ảnh
def preprocess_image(img_path, target_size=(160,160)):
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size).astype(np.float32)
    img = (img / 127.5) - 1.0
    # thêm batch dim
    return np.expand_dims(img, axis=0)  # shape (1, H, W, C)

# 4) Warm‑up (build graph, load weights vào cache)
sample = preprocess_image(img_paths[0])
_ = emb_model(sample, training=False)

# 5) (Tuỳ chọn) Wrap inference vào tf.function để chạy nhanh hơn
infer_fn = tf.function(lambda x: emb_model(x, training=False),
                       input_signature=[tf.TensorSpec([1,160,160,3], tf.float32)])
_ = infer_fn(sample)  # warm‑up graph

# 6) Đo thời gian per‑image
total_time = 0.0
for p in img_paths:
    img = preprocess_image(p)
    start = time.perf_counter()
    _ = infer_fn(img)
    end = time.perf_counter()
    total_time += (end - start)

n_images = len(img_paths)
print(f"Processed {n_images} images in {total_time:.4f} seconds")
print(f"Average per image: {total_time / n_images:.4f} seconds")

Processed 424 images in 1.2917 seconds
Average per image: 0.0030 seconds
